# One-to-Many and Many-to-Many Joins - Lab

## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:
* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='images/Database-Schema.png' width="600">

## Connect to the Database

Include the relevant imports, then connect to the database located at `data.sqlite`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [9]:
schemas = cur.execute("""SELECT name, sql FROM sqlite_master WHERE type = 'table'""").fetchall()

for schema in schemas:
    print(f"{schema[0]} schema:")
    print(schema[1], '\n')

productlines schema:
CREATE TABLE `productlines` (`productLine`, `textDescription`, `htmlDescription`, `image`) 

offices schema:
CREATE TABLE "offices" (
	"officeCode"	INTEGER,
	"city"	TEXT,
	"phone"	TEXT,
	"addressLine1"	TEXT,
	"addressLine2"	TEXT,
	"state"	TEXT,
	"country"	TEXT,
	"postalCode"	TEXT,
	"territory"	TEXT
) 

customers schema:
CREATE TABLE "customers" (
	"customerNumber"	INTEGER,
	"customerName"	TEXT,
	"contactLastName"	TEXT,
	"contactFirstName"	TEXT,
	"phone"	TEXT,
	"addressLine1"	TEXT,
	"addressLine2"	TEXT,
	"city"	TEXT,
	"state"	TEXT,
	"postalCode"	TEXT,
	"country"	TEXT,
	"salesRepEmployeeNumber"	INTEGER,
	"creditLimit"	NUMERIC
) 

employees schema:
CREATE TABLE "employees" (
	"employeeNumber"	INTEGER,
	"lastName"	TEXT,
	"firstName"	TEXT,
	"extension"	TEXT,
	"email"	TEXT,
	"officeCode"	INTEGER,
	"reportsTo"	INTEGER,
	"jobTitle"	TEXT
) 

orderdetails schema:
CREATE TABLE "orderdetails" (
	"orderNumber"	INTEGER,
	"productCode"	TEXT,
	"quantityOrdered"	INTEGER,
	"priceEac

## Employees and Their Offices (a One-to-One Join)

Select all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [20]:
# Your code here
q = """
SELECT
    e.firstName AS first_name,
    e.lastName AS last_name,
    o.city AS city,
    CASE
        WHEN o.state != '' THEN o.state
        ELSE 'No state provided'
    END AS state
FROM
    employees AS e
    LEFT JOIN offices AS o
        USING (officeCode)
ORDER BY
    first_name, last_name;"""
for first, last, city, state in cur.execute(q).fetchall():
    print(f"{first} {last} | {city} | {state}")
pd.read_sql(q, conn)

Andy Fixter | Sydney | No state provided
Anthony Bow | San Francisco | CA
Barry Jones | London | No state provided
Diane Murphy | San Francisco | CA
Foon Yue Tseng | NYC | NY
George Vanauf | NYC | NY
Gerard Bondur | Paris | No state provided
Gerard Hernandez | Paris | No state provided
Jeff Firrelli | San Francisco | CA
Julie Firrelli | Boston | MA
Larry Bott | London | No state provided
Leslie Jennings | San Francisco | CA
Leslie Thompson | San Francisco | CA
Loui Bondur | Paris | No state provided
Mami Nishi | Tokyo | Chiyoda-Ku
Martin Gerard | Paris | No state provided
Mary Patterson | San Francisco | CA
Pamela Castillo | Paris | No state provided
Peter Marsh | Sydney | No state provided
Steve Patterson | Boston | MA
Tom King | Sydney | No state provided
William Patterson | Sydney | No state provided
Yoshimi Kato | Tokyo | Chiyoda-Ku


,first_name,last_name,city,state
0,Andy,Fixter,Sydney,No state provided
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,No state provided
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,No state provided
7,Gerard,Hernandez,Paris,No state provided
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and Their Orders (a One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [22]:
# Your code here
q = """
SELECT
    c.contactFirstName AS first_name,
    c.contactLastName AS last_name,
    o.orderNumber AS order_number,
    o.orderDate AS order_date,
    o.status AS status
FROM customers AS c
    JOIN orders AS o
        USING(customerNumber)
    """
pd.read_sql(q, conn)

,first_name,last_name,order_number,order_date,status
0,Carine,Schmitt,10123,2003-05-20,Shipped
1,Carine,Schmitt,10298,2004-09-27,Shipped
2,Carine,Schmitt,10345,2004-11-25,Shipped
3,Jean,King,10124,2003-05-21,Shipped
4,Jean,King,10278,2004-08-06,Shipped
...,...,...,...,...,...
321,Valarie,Franco,10243,2004-04-26,Shipped
322,Tony,Snowden,10138,2003-07-07,Shipped
323,Tony,Snowden,10179,2003-11-11,Cancelled
324,Tony,Snowden,10360,2004-12-16,Shipped


## Customers and Their Payments (Another One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [26]:
# Your code here
q = """
SELECT 
    c.contactFirstName AS "First Name",
    c.contactLastName AS "Last Name",
    p.paymentDate AS "Payment Date",
    p.amount AS "Amount Paid ($)"
    
FROM customers AS c
    JOIN payments as p
        USING (customerNumber)
ORDER BY
    p.amount DESC"""

pd.read_sql(q, conn)

,First Name,Last Name,Payment Date,Amount Paid ($)
0,Diego,Freyre,2005-03-18,120166.58
1,Diego,Freyre,2004-12-31,116208.40
2,Susan,Nelson,2003-08-15,111654.40
3,Eric,Natividad,2003-12-26,105743.00
4,Susan,Nelson,2005-03-05,101244.59
...,...,...,...,...
268,Carine,Schmitt,2004-12-18,1676.14
269,Pascale,Cartrain,2003-04-19,1627.56
270,Jonas,Bergulfsen,2003-10-28,1491.38
271,Pascale,Cartrain,2003-08-22,1128.20


## Orders, Order Details, and Product Details (a Many-to-Many Join)

Select all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [29]:
# Your code here
q = """
SELECT
    c.contactFirstName AS "First Name",
    c.contactLastName AS "Last Name",
    p.productName AS "Product",
    od.quantityOrdered AS "Quantity Ordered",
    o.orderDate AS "Order Date"

FROM customers AS c
    JOIN orders AS o
        USING (customerNumber)
    JOIN orderdetails AS od
        USING (orderNumber)
    JOIN products AS p
        USING (productCode)

ORDER BY
    o.orderDate"""

pd.read_sql(q, conn)

,First Name,Last Name,Product,Quantity Ordered,Order Date
0,Dorothy,Young,1917 Grand Touring Sedan,30,2003-01-06
1,Dorothy,Young,1911 Ford Town Car,50,2003-01-06
2,Dorothy,Young,1932 Alfa Romeo 8C2300 Spider Sport,22,2003-01-06
3,Dorothy,Young,1936 Mercedes Benz 500k Roadster,49,2003-01-06
4,Roland,Keitel,1932 Model A Ford J-Coupe,25,2003-01-09
...,...,...,...,...,...
2991,Diego,Freyre,1958 Setra Bus,49,2005-05-31
2992,Diego,Freyre,1940 Ford Pickup Truck,54,2005-05-31
2993,Diego,Freyre,1939 Cadillac Limousine,26,2005-05-31
2994,Diego,Freyre,1996 Peterbilt 379 Stake Bed with Outrigger,44,2005-05-31


## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!